In [1]:
import numpy as np
import matplotlib.pyplot as plt
from evalf import evalf
from dt_compare import dt_compare
from scipy.integrate import odeint
from simple_trap import simple_trap
from simple_trap import simple_matrix_free_trap
from euler import euler
from trapezoidal import rk_integrator
from trapezoidal import expand_params
from deathrate import a
%load_ext autoreload
%autoreload 2


def better_imshow(row_vals, col_vals, data, ax=None):
    #row_vals and col_vals are the data that was swept over
    assert len(row_vals) == data.shape[0], 'length of row_vals does not match shape of data'
    assert len(col_vals) == data.shape[1], 'length of col_vals does not match shape of data'
    if ax == None:
        plt.imshow(data, origin='lower', extent=[col_vals[0], col_vals[-1], row_vals[0], row_vals[-1]], aspect='auto')
    else:
        ax.imshow(data, origin='lower', extent=[col_vals[0], col_vals[-1], row_vals[0], row_vals[-1]], aspect='auto')

In [2]:
dt = [1e-3, 5e-4, 1e-4, 5e-5]
t_final = 0.1
L_list = np.linspace(0,1e-3, 1000)

S0 = 1 #initial supersaturation
V0 = 1 #initial volume
n0 = np.zeros_like(L_list) #initial population density distribution

p = {"L_list" : L_list, #discretization bins
     'E' :  1e-7, #evaporation rate
     'rho' : 1200, # density of the crystal
     'k_v' : 1, #goes in N_C, volumetric shape factor
     'k_g' : 1e-3, #growth rate constant m/s
     'g' : 1, #power constant for growth
     'k_N' : 1e5, #nucleation rate constant 
     'alpha' :2, #power constant for nucleation
     'Breakage': True, #toggle breakage for debug
     'weno': False  # use weno or not for differentiation w.r.t. L
     }

p_expand = expand_params(p)

#integrate the equations
x0 = np.hstack([S0, V0, n0])
for f in [euler, rk_integrator]:
     error, t = dt_compare(x0, dt, p_expand, f, t_final)


Errors for different time discretizations are: [0.04589547905294074, 0.019322511850179514, 0.0007938372628671066]
Time taken for different time discretizations are: <module 'time' (built-in)>


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Errors for different time discretizations are: [0.008050200011452551, 0.0043094011865327285, 0.002003019905884553]
Time taken for different time discretizations are: <module 'time' (built-in)>


In [6]:
print(error)
print(t)

[0.008050200011452551, 0.0043094011865327285, 0.002003019905884553]
[2.006974220275879, 3.925609827041626, 21.11407208442688]


In [4]:
tlist = np.linspace(0,30,1000) 


S0 = 1 #initial supersaturation
V0 = 1 #initial volume
n0 = np.zeros_like(L_list) #initial population density distribution

#parameters
p = {"L_list" : L_list, #discretization bins
     'E' :  1e-6, #evaporation rate (m^3/s)
     'rho' : 1200, # density of the crystal
     'k_v' : 1, #goes in N_C, volumetric shape factor
     'k_g' : 1e-3, #growth rate constant m/s
     'g' : 1, #power constant for growth
     'k_N' : 1e3, #nucleation rate constant 
     'alpha' :2, #power constant for nucleation
     'Breakage': True #toggle breakage for debug
     }


#integrate the equations
x = np.hstack([S0, V0, n0])
x_t = odeint(evalf, y0=x, t=tlist, args=(p,None))


#plot results
n_t = x_t[:,2:]
better_imshow(L_list, tlist, n_t.T)
plt.ylabel('L (particle size)')
plt.xlabel('time')
plt.colorbar()
plt.show()

plt.plot(tlist, x_t[:,0]) 
plt.title('supersaturation')
plt.xlabel('t')
plt.show()

plt.plot(tlist, x_t[:,1]) 
plt.title('volume')
plt.xlabel('t')

TypeError: evalf() takes 2 positional arguments but 4 were given

In [ ]:
#linecuts

n_t = x_t[:,2:]

for t_ind in [0,50, 500, 1000]:
    plt.plot(L_list*1e3, n_t[t_ind,:], label='t_ind = '+str(t_ind))
    
plt.xlabel('L (mm)')
plt.title('linecuts of n(t)')
plt.legend(frameon=False)